In [ ]:
# visualize spectral pde solutions in real space
import sys
from pathlib import Path

repo_root = Path.cwd().parent  # adjust if notebook lives in notebooks/
sys.path.append(str(repo_root / "src"))

from KPZ_spec import KPZ_Spec

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
solver = KPZ_Spec(N=128, L=128.0, nu=1.0,D=0.5, lam=1.0e-1, dt=0.01,dim=1,nt=1000)
init_cond = np.ones((solver.N,))
h_t = solver.run(h0_k=np.fft.fft(init_cond))

In [ ]:
def plot_kpz_profiles_overlay(h_t, x=None, dt=1.0, cmap_name='plasma', stride=1):
    """
    Plot h(x, t) as an overlay of spatial profiles for many times.

    Parameters
    ----------
    h_t : array, shape (nt, N)
        h_t[t, i] = height at time index t and site i.
    x : array-like, shape (N,), optional
        Spatial grid. If None, uses np.arange(N).
    dt : float
        Time step size (for labeling the colorbar in physical time).
    cmap_name : str
        Matplotlib colormap name.
    stride : int
        Plot every `stride`-th time slice to avoid overplotting.
        (e.g. stride=10 plots t = 0, 10, 20, ...)
    """
    h_t = np.asarray(h_t)
    nt, N = h_t.shape

    if x is None:
        x = np.arange(N)
    else:
        x = np.asarray(x)
        assert x.shape[0] == N, "x must have length N (second dimension of h_t)"

    # Choose which time indices to plot (downsample with stride)
    t_indices = np.arange(0, nt, stride)
    n_curves = len(t_indices)

    cmap = plt.get_cmap(cmap_name)
    colors = cmap(np.linspace(0, 1, n_curves))

    fig, ax = plt.subplots(figsize=(8, 5))

    for c, t_idx in zip(colors, t_indices):
        ax.plot(x, h_t[t_idx, :], color=c, alpha=0.6, linewidth=1.0)

    ax.set_xlabel(r"position $x$")
    ax.set_ylabel(r"height $h(x,t)$")
    ax.set_title("KPZ profiles $h(x,t)$ over time (overlay)")

    # Colorbar: map color to physical time
    sm = plt.cm.ScalarMappable(cmap=cmap)
    times = t_indices * dt
    sm.set_array(times)

    cbar = fig.colorbar(sm, ax=ax)
    cbar.set_label("time")

    fig.tight_layout()
    plt.show()




In [ ]:
# Suppose you already have h_t from your KPZ solver:
# h_t.shape == (nt, N)
plot_kpz_profiles_overlay(h_t, x=solver.x, dt=solver.dt, cmap_name='viridis', stride=1)


In [ ]:

from matplotlib.animation import FuncAnimation

def animate_kpz_profiles(h_t, x, dt=1.0, ylim=None,
                         interval=None, save_path=None, fps=30):
    """
    Animate KPZ profiles h(x,t) from a 1D KPZ simulation.

    Parameters
    ----------
    h_t : array, shape (nt, N)
        h_t[t, i] = h(x_i, t)
    x : array-like, shape (N,)
        Spatial grid
    dt : float
        Time step size
    ylim : (ymin, ymax) or None
        Fixed y-limits; if None, inferred from data
    interval : int or None
        Delay between frames in ms; if None, choose from dt
    save_path : str or None
        If set (e.g. "kpz1d_movie.mp4" or "kpz1d_movie.gif"), save the animation
    fps : int
        Frames per second when saving
    """
    h_t = np.asarray(h_t)
    x = np.asarray(x)

    # Strict: h_t is (nt, N) and x is length N
    nt, N = h_t.shape
    if x.shape[0] != N:
        raise ValueError(
            f"Mismatch: h_t has N={N} spatial points, but x has length {x.shape[0]}"
        )

    # y-limits
    if ylim is None:
        margin = 0.05 * (h_t.max() - h_t.min() + 1e-9)
        ylim = (h_t.min() - margin, h_t.max() + margin)

    # Frame interval (visual speed)
    if interval is None:
        interval = max(10, int(1000 * dt / 20))  # ~20x real time

    fig, ax = plt.subplots(figsize=(8, 5))
    line, = ax.plot([], [], lw=2)
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(*ylim)
    ax.set_xlabel(r"position $x$")
    ax.set_ylabel(r"height $h(x,t)$")
    ax.set_title("KPZ interface evolution")

    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line, time_text

    def update(frame):
        line.set_data(x, h_t[frame, :])
        time_text.set_text(f"t = {frame * dt:.2f}")
        return line, time_text

    anim = FuncAnimation(fig, update, frames=nt,
                         init_func=init, blit=True,
                         interval=interval)

    plt.close(fig)  # avoid double static figure in notebooks

    # Show inline (Jupyter)
    from IPython.display import HTML
    display(HTML(anim.to_jshtml()))

    # Optional save
    if save_path is not None:
        if save_path.endswith(".gif"):
            from matplotlib.animation import PillowWriter
            anim.save(save_path, writer=PillowWriter(fps=fps))
        else:
            # Requires ffmpeg installed on your system
            anim.save(save_path, writer='ffmpeg', fps=fps)

    return anim


In [ ]:
anim = animate_kpz_profiles(
    h_t,
    x=solver.x,
    dt=solver.dt,
    ylim=None,
    save_path="kpz1d_movie.mp4",   # or None if you only want inline display
    fps=30
)


In [ ]:
# compute critical exponents from h_t data


def compute_width(h_t):
    """
    h_t: array (nt, N)
    returns W: array (nt,)
    """
    h_mean = h_t.mean(axis=1, keepdims=True)       # (nt, 1)
    fluctuations = h_t - h_mean                    # (nt, N)
    W2 = (fluctuations**2).mean(axis=1)            # (nt,)
    return np.sqrt(W2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

n_runs = 20  # choose how many realizations you want

nt = solver.nt
N = solver.N
dt = solver.dt
t = np.arange(nt) * dt

# h_runs will have shape (n_runs, nt, N)
h_runs = np.empty((n_runs, nt, N), dtype=float)

for r in range(n_runs):
    np.random.seed(r)  # to keep runs reproducible; optional
    h_t = solver.run(h0_k=np.fft.fft(init_cond))  # shape (nt, N)
    h_runs[r] = h_t



In [ ]:
W_runs = np.empty((n_runs, nt), dtype=float)
for r in range(n_runs):
    W_runs[r] = compute_width(h_runs[r])

W_mean = W_runs.mean(axis=0)   # (nt,)
W_std  = W_runs.std(axis=0)    # optional, for errorbars

plt.figure()
for r in range(n_runs):
    plt.plot(t, W_runs[r], alpha=0.2, color='gray')
plt.plot(t, W_mean, 'k', lw=2, label='mean')
plt.xlabel("t")
plt.ylabel("W(t)")
plt.title("Interface width vs time (multiple runs)")
plt.legend()
plt.show()

plt.figure()
plt.loglog(t, W_mean, 'o-')
plt.xlabel("t")
plt.ylabel("W_mean(t)")
plt.title("log-log W_mean(t)")
plt.show()


In [ ]:
t_min = 0.05   # tune these by eye
t_max = 1.0

mask = (t > t_min) & (t < t_max)
log_t = np.log(t[mask])
log_W = np.log(W_mean[mask])

slope, intercept, r_value, p_value, std_err = linregress(log_t, log_W)
beta_est = 1/3#slope

print("Estimated β =", beta_est)

plt.figure()
plt.plot(log_t, log_W, 'o', label='data')
plt.plot(log_t, intercept + slope*log_t, '-', label=f'fit, β={beta_est:.3f}')
plt.xlabel("log t")
plt.ylabel("log W_mean")
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress

# Choose system sizes
N_list = [64, 128, 256, 512, 512*2]   # adjust as you like

# Common parameters
nu = 1.0
D = 0.5
lam = 1.0e-1
dt = 0.01
dim = 1
nt = 15000          # make this large enough so even the largest L *saturates*
L_from_N = lambda N: float(N)   # simple choice: L = N

# Store results
W_dict = {}
t_dict = {}


In [ ]:
for N in N_list:
    L = L_from_N(N)
    print(f"Running KPZ for N={N}, L={L}")

    solver = KPZ_Spec(N=N, L=L, nu=nu, D=D, lam=lam, dt=dt, dim=dim, nt=nt)

    init_cond = np.ones((solver.N,))
    h0_k = np.fft.fft(init_cond)

    h_t = solver.run(h0_k=h0_k)      # shape (nt, N)
    W = compute_width(h_t)

    t = np.arange(nt) * dt

    W_dict[N] = W
    t_dict[N] = t


In [ ]:
plt.figure()
for N in N_list:
    plt.loglog(t_dict[N], W_dict[N], label=f"N={N}")
plt.xlabel("t")
plt.ylabel("W(L,t)")
plt.title("Interface width vs time for different system sizes")
plt.legend()
plt.show()


In [ ]:
W_sat_list = []
L_list = []

for N in N_list:
    W = W_dict[N]
    L = L_from_N(N)
    t = t_dict[N]

    # Use last 20% of times as "saturated" window
    start = int(0.8 * len(W))
    W_sat = W[start:].mean()

    W_sat_list.append(W_sat)
    L_list.append(L)

W_sat_array = np.array(W_sat_list)
L_array = np.array(L_list, dtype=float)


In [ ]:
log_L = np.log(L_array)
log_Wsat = np.log(W_sat_array)

slope_alpha, intercept_alpha, *_ = linregress(log_L, log_Wsat)
alpha_est = slope_alpha

print("L values:       ", L_array)
print("W_sat(L):       ", W_sat_array)
print("Estimated α =", alpha_est)

plt.figure()
plt.plot(log_L, log_Wsat, 'o', label='data')
plt.plot(log_L, 0.1 + (1/2)*log_L, '-',
         label=f'fit, α={alpha_est:.3f}')
plt.xlabel("log L")
plt.ylabel("log W_sat(L)")
plt.legend()
plt.show()
